# 5G NR TDD UDC5 Pattern Test

This notebook generates a 5G NR TDD waveform using UDC5 pattern:
- **Slots**: `[0,0,0,0,0,0,0,2,1,1,0,0,0,0,0,0,0,2,1,1]`
- **Special Slot**: `[0,0,0,0,0,0,2,2,2,3,1,1,1,1]`

Where:
- `0` = DL slot/symbol
- `1` = UL slot/symbol
- `2` = Guard period (special slot only)
- `3` = Switch point (special slot only)


In [ ]:
import pyPhyNR as pynr
from pyPhyNR.core.signal_builder import NRSignalBuilder


In [ ]:
# UDC5 TDD Configuration
TDD_SLOT_PATTERN = [0,0,0,0,0,0,0,2,1,1,0,0,0,0,0,0,0,2,1,1]  # 20 slots
SPECIAL_SLOT_SYMBOL_PATTERN = [0,0,0,0,0,0,2,2,2,3,1,1,1,1]  # 14 symbols

print("📊 TDD Configuration:")
print(f"   Slot Pattern: {TDD_SLOT_PATTERN}")
print(f"   Special Slot Symbols: {SPECIAL_SLOT_SYMBOL_PATTERN}")

# Count slot types
dl_slots = TDD_SLOT_PATTERN.count(0)
ul_slots = TDD_SLOT_PATTERN.count(1)
special_slots = TDD_SLOT_PATTERN.count(2)

print(f"\n📈 Slot Distribution:")
print(f"   DL Slots: {dl_slots}")
print(f"   UL Slots: {ul_slots}")
print(f"   Special Slots: {special_slots}")

# Count special slot symbol types
dl_symbols = SPECIAL_SLOT_SYMBOL_PATTERN.count(0)
ul_symbols = SPECIAL_SLOT_SYMBOL_PATTERN.count(1)
guard_symbols = SPECIAL_SLOT_SYMBOL_PATTERN.count(2)
switch_symbols = SPECIAL_SLOT_SYMBOL_PATTERN.count(3)

print(f"\n📈 Special Slot Symbol Distribution:")
print(f"   DL Symbols: {dl_symbols}")
print(f"   UL Symbols: {ul_symbols}")
print(f"   Guard Symbols: {guard_symbols}")
print(f"   Switch Symbols: {switch_symbols}")


In [ ]:
# Create signal builder for NR20, SCS30
signal = NRSignalBuilder(bandwidth_mhz=20, numerology=1, cell_id=1)
signal.configure_carrier(sample_rate=23.04e6, fft_size=768).initialize_grid()


In [ ]:
# Add PDSCH to DL slots (full 14 symbols)
dl_slots = [i for i, slot_type in enumerate(TDD_SLOT_PATTERN) if slot_type == 0]
print(f"Adding PDSCH to DL slots: {dl_slots}")

signal.add_pdsch(
    start_rb=0,
    num_rb=51,
    start_symbol=0,
    num_symbols=14,
    slot_pattern=dl_slots,
    modulation="QAM256",
    power=100.0
).add_dmrs(
    dmrs_positions=[2, 11],
    clear_full_symbol=False,
    subcarrier_pattern="even",
    power_offset_db=0.0
)


In [ ]:
# Add PDSCH to special slots (only DL symbols)
special_slots = [i for i, slot_type in enumerate(TDD_SLOT_PATTERN) if slot_type == 2]

# Count DL symbols in special slot
dl_symbols_in_special = SPECIAL_SLOT_SYMBOL_PATTERN.count(0)
print(f"   DL symbols in special slot: {dl_symbols_in_special}")

if special_slots:
    signal.add_pdsch(
        start_rb=0,
        num_rb=51,
        start_symbol=0,
        num_symbols=dl_symbols_in_special,
        slot_pattern=special_slots,
        modulation="QAM256",
        power=100.0
    ).add_dmrs(
        dmrs_positions=[2],
        clear_full_symbol=False,
        subcarrier_pattern="even",
        power_offset_db=0.0
    )


In [ ]:

print("\nPlotting resource grid...")
pynr.utils.plot_grid_dl(signal.carrier_config, signal.grid)

print("\nGenerating waveform...")
iq_samples = signal.generate_signal()
pynr.utils.plot_time_domain(iq_samples, signal.carrier_config)
pynr.utils.plot_frequency_domain(iq_samples, signal.carrier_config)
